In [15]:
import matplotlib
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import r2_score

In [16]:
df = pd.read_csv('data1009train.csv',encoding='UTF-8-Sig')
x = torch.tensor(df.iloc[:,1:-1].to_numpy()).to(torch.float32)
y = torch.tensor(df.iloc[:,-1].to_numpy()).to(torch.float32)
x.shape

torch.Size([2164, 168])

In [17]:
class stockDataSetter(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [18]:
Batchsize = x.shape[0]//20
train_set = stockDataSetter(x, y)
train_loader = DataLoader(train_set, Batchsize,shuffle=True, pin_memory=True)

In [19]:
import torch
import torch.nn.functional as F   

class My_model(torch.nn.Module): 
    def __init__(self, input_dim):
        super(My_model, self).__init__()
        self.layers = torch.nn.Sequential(
            nn.Linear(input_dim, 168),
            nn.ReLU(),
            nn.Linear(168, 1)
            )
        
    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

net = My_model(input_dim=x.shape[1])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.RAdam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-5, weight_decay=0, foreach=True)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False, foreach=None, maximize=False, capturable=False)
loss_function = torch.nn.MSELoss(reduction='mean')

print(net)  # net 的结构

My_model(
  (layers): Sequential(
    (0): Linear(in_features=168, out_features=168, bias=True)
    (1): ReLU()
    (2): Linear(in_features=168, out_features=1, bias=True)
  )
)


In [20]:
#訓練
# checkpoint = torch.load('model_state_dict1.pt')
# net.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']
net = net.train()


best_loss = math.inf
patience_bestloss = 0

bestlossMean = math.inf
loss_50_list = []
patience_meanloss = 0
for t in range(1000000):
    
    x, y = next(iter(train_loader))
    
    prediction = net(x).to(device)
    loss = loss_function(prediction,y).to(device)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    

    loss_50_list.append(loss.data.numpy())
    # 接着上面来
    if t % 50 == 0:
        lossMean = sum(loss_50_list)/len(loss_50_list)
        print('================================================')
        print(f'第:{t}輪', 'loss: ',loss.data.numpy())
        print(f'前50輪loss平均: {lossMean}')
        print(f'目前最佳loss: {best_loss}')
        print(f'目前最佳平均loss: {bestlossMean}')
        print('patience_meanloss(最大1000): ',patience_meanloss )
        print('patience_bestloss(最大50000): ',patience_bestloss )
        print('================================================')
        loss_50_list = []
        if lossMean <=bestlossMean:
            bestlossMean = lossMean
            patience_meanloss = 0
        else:
            patience_meanloss+=1
        
        if patience_meanloss == 10000:
            print('平均loss已經10000個batch未下降')
            break
        prelossMean = lossMean
        
    if loss<best_loss and prelossMean == bestlossMean:
        patience_bestloss = 0
        best_loss = loss
        torch.save(net, 'Save_File1.pth')
        torch.save({
            'epoch': t,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'model_state_dict1.pt')
        print(f'儲存model, loss={loss}, 前50輪平均loss={prelossMean}')
    else:
        patience_bestloss += 1
    
    if patience_bestloss >= 50000:
        print('loss已經訓練50000個batch未下降')
        break
    

第:0輪 loss:  24366.967
前50輪loss平均: 24366.966796875
目前最佳loss: inf
目前最佳平均loss: inf
patience_meanloss(最大1000):  0
patience_bestloss(最大50000):  0
儲存model, loss=24366.966796875, 前50輪平均loss=24366.966796875
儲存model, loss=19793.61328125, 前50輪平均loss=24366.966796875
儲存model, loss=13246.294921875, 前50輪平均loss=24366.966796875
儲存model, loss=13228.6875, 前50輪平均loss=24366.966796875
儲存model, loss=11557.2412109375, 前50輪平均loss=24366.966796875
儲存model, loss=10353.1787109375, 前50輪平均loss=24366.966796875
第:50輪 loss:  20258.34
前50輪loss平均: 83605.50853515624
目前最佳loss: 10353.1787109375
目前最佳平均loss: 24366.966796875
patience_meanloss(最大1000):  0
patience_bestloss(最大50000):  0
第:100輪 loss:  11288.613
前50輪loss平均: 18728.758671875
目前最佳loss: 10353.1787109375
目前最佳平均loss: 24366.966796875
patience_meanloss(最大1000):  1
patience_bestloss(最大50000):  50
第:150輪 loss:  34010.58
前50輪loss平均: 19548.1285546875
目前最佳loss: 10353.1787109375
目前最佳平均loss: 18728.758671875
patience_meanloss(最大1000):  0
patience_bestloss(最大50000):  100
第:200輪 l

In [8]:
#檢查
checkpoint = torch.load('model_state_dict1.pt')
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
net = net.eval()

prediction = net(x).to(device)
loss = loss_function(prediction,y).to(device)

pred,actual = pd.DataFrame(prediction.data.numpy()), pd.DataFrame(y.data.numpy())
compare = pd.DataFrame(columns=['actual','predict'])
compare['actual'],compare['predict'] = y.data.numpy(), prediction.data.numpy()
compare = compare.T

RuntimeError: Error(s) in loading state_dict for My_model:
	Unexpected key(s) in state_dict: "layers.4.weight", "layers.4.bias", "layers.6.weight", "layers.6.bias", "layers.8.weight", "layers.8.bias", "layers.10.weight", "layers.10.bias", "layers.12.weight", "layers.12.bias", "layers.14.weight", "layers.14.bias", "layers.16.weight", "layers.16.bias". 
	size mismatch for layers.2.weight: copying a param with shape torch.Size([148, 168]) from checkpoint, the shape in current model is torch.Size([1, 168]).
	size mismatch for layers.2.bias: copying a param with shape torch.Size([148]) from checkpoint, the shape in current model is torch.Size([1]).

In [ ]:
ax = plt.gca()
print('loss: ',loss.data.numpy())
compare.T.plot(kind='line',y ='predict',ax=ax,color='black')
compare.T.plot(kind='line',y ='actual',ax=ax,color='red')
plt.show()
compare
# Display all model layer weights
# for name, para in net.named_parameters():
#     print('{}: {}'.format(name, para.shape))

In [201]:
r2_score(compare.T['actual'],compare.T['predict'])

0.9997916465926044